## Cluster Toronto Neighborhoods

I will use the csv I saved from the previous exercise (getting Lat and Long for Toronto neighborhoods) and use the FourSquare calls to retrieve venue types for each neighborhood, then use k-means to cluster the neighborhoods into 4 clusters and try to describe the clusters by their venue types.

### First, import the needed libraries

In [4]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Second, get the csv I created in the previous exercise.

In [8]:
df_Neighborhoods = pd.read_csv('TorontoNeighborhoodsLatLon.csv',)
print(df_Neighborhoods.shape)
df_Neighborhoods.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
# @hidden_cell

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret

VERSION = '20180605'


#CLIENT_ID = 'your-client-ID' # your Foursquare ID
#CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Add the function to get the venues near a specific latlon

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
# type your answer here
# define the new dataframe
df_Toronto_Venues = getNearbyVenues(names=df_Neighborhoods['Neighborhood'], latitudes=df_Neighborhoods['Latitude'], longitudes=df_Neighborhoods['Longitude'])
df_Toronto_Venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


How many Toronto venues do I have?

In [26]:
df_Toronto_Venues.shape

(2128, 7)

Group them by neighborhood and ...
## Drop Neighborhoods with too few venues
I am going to drop neighborhoods that have fewer than 20 venues because I think they will have too few to categorize by venue type:

In [33]:
df_Neighborhoods_To_Use = df_Toronto_Venues.groupby('Neighborhood').count().sort_values('Neighborhood Latitude', ascending=False)[0:31]

df_Neighborhoods_To_Use.reset_index(level=0, inplace=True) 
top_neighborhoods = df_Neighborhoods_To_Use['Neighborhood']

df_Venues_To_Use = df_Toronto_Venues.loc[df_Toronto_Venues['Neighborhood'].isin(top_neighborhoods)]
print(df_Venues_To_Use.shape)

df_Venues_To_Use.head()

(1692, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
10,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
11,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


How many unique categories of venues do we have?

In [34]:
print('There are {} uniques categories.'.format(len(df_Venues_To_Use['Venue Category'].unique())))

There are 223 uniques categories.


## Time to Analyze the Neighborhoods

In [48]:
# one hot encoding
df_Toronto_Onehot = pd.get_dummies(df_Venues_To_Use[['Venue Category']], prefix="", prefix_sep="")

# rename 'Neighborhood' venue category
df_Toronto_Onehot.rename(columns={'Neighborhood': 'A Neighborhood'}, inplace=True)

# add neighborhood column back to dataframe
df_Toronto_Onehot['Neighborhood'] = df_Venues_To_Use['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = ['Neighborhood'] + list(df_Toronto_Onehot.columns[:-1])
df_Toronto_Onehot = df_Toronto_Onehot[fixed_columns]

df_Toronto_Onehot.head(2)

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,A Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
7,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [49]:
# what is the shape of the resulting dummied DataFrame?
df_Toronto_Onehot.shape

(1692, 224)

### Group the DataFrame by Neighborhood and examine

In [50]:
df_Toronto_Grouped = df_Toronto_Onehot.groupby('Neighborhood').mean().reset_index()
df_Toronto_Grouped.head(20)

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,A Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.086957,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.043478,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.086957,0.0,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000

Add the function to retrieve the most common venues for the neighborhood

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_Toronto_Grouped['Neighborhood']

for ind in np.arange(df_Toronto_Grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_Toronto_Grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Ice Cream Shop,Pharmacy,Pizza Place,Deli / Bodega,Mobile Phone Shop,Middle Eastern Restaurant,Restaurant
1,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Butcher,Indian Restaurant,Sushi Restaurant,Restaurant,Liquor Store,Spa,Pizza Place
2,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Bakery,Cheese Shop,Pharmacy,Liquor Store
3,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Salad Place,Middle Eastern Restaurant,Thai Restaurant,Burger Joint,Miscellaneous Shop


## Cluster the Toronto Neighborhoods

Using k-means, cluster the Toronto Neighborhoods into 4 clusters. See what the commonalities are.

In [63]:
# set number of clusters: I a setting to 4
kclusters = 4

df_Toronto_grouped_clustering = df_Toronto_Grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([1, 2, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [64]:
# checking the cluster labels to make sure I see 0,1,2,3
kmeans.labels_[0:50] 

array([1, 2, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 2, 1, 0, 1])

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_Toronto_merged = df_Venues_To_Use

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
df_Toronto_merged = df_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_Toronto_merged.head() 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
8,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
9,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
10,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
11,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant


## Map the clusters using the folium library.

In [67]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_Toronto_merged['Neighborhood Latitude'], df_Toronto_merged['Neighborhood Longitude'], df_Toronto_merged['Neighborhood'], df_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the Cluster Characteristics

### Cluster 1
Looks like a hip neighborhood with coffeeshops, bars, inexpensive and trendy restaurants. Probably young people live in these Cluster 1 neighborhoods.

In [77]:
df_Toronto_merged.loc[df_Toronto_merged['Cluster Labels'] == 0]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1073,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Reebok Crossfit Liberty Village,43.637036,-79.424802,Gym,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1074,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Starbucks,43.639090,-79.427622,Coffee Shop,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1075,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Pharmacy,43.638090,-79.431810,Bar,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1076,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Caffino,43.639021,-79.425289,Italian Restaurant,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1077,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Louie Craft Coffee,43.639284,-79.425620,Coffee Shop,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1078,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,SCHOOL Restaurant,43.637775,-79.424297,Breakfast Spot,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1079,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Pet Valu,43.639089,-79.426828,Pet Store,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1080,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Structube,43.639836,-79.423642,Furniture / Home Store,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1081,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,The Abbott,43.637996,-79.430717,Café,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place
1082,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Mike's Liberty Grill,43.637505,-79.424650,Breakfast Spot,0,Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Pet Store,Climbing Gym,Restaurant,Burrito Place


### Cluster Two
A less population dense version of CLuster One - I say this because of the parks. Probably young families live in these neighborhoods.

In [76]:
df_Toronto_merged.loc[df_Toronto_merged['Cluster Labels'] == 1]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
10,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
11,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
12,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
13,"Regent Park, Harbourfront",43.654260,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
14,"Regent Park, Harbourfront",43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
15,"Regent Park, Harbourfront",43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant
16,"Regent Park, Harbourfront",43.654260,-79.360636,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop,1,Coffee Shop,Park,Pub,Bakery,Café,Theater,Breakfast Spot,Historic Site,Hotel,Italian Restaurant


### Cluster Three

In [75]:
df_Toronto_merged.loc[df_Toronto_merged['Cluster Labels'] == 2]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
566,Thorncliffe Park,43.705369,-79.349372,Costco,43.707051,-79.348093,Warehouse Store,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
567,Thorncliffe Park,43.705369,-79.349372,Fit4Less,43.705689,-79.346018,Gym,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
568,Thorncliffe Park,43.705369,-79.349372,Iqbal Kebab & Sweet Centre,43.705923,-79.351521,Indian Restaurant,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
569,Thorncliffe Park,43.705369,-79.349372,Bikram Yoga East York,43.705450,-79.351448,Yoga Studio,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
570,Thorncliffe Park,43.705369,-79.349372,Hero Certified Burgers,43.705511,-79.347064,Burger Joint,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
571,Thorncliffe Park,43.705369,-79.349372,Shoppers Drug Mart,43.705810,-79.347044,Pharmacy,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
572,Thorncliffe Park,43.705369,-79.349372,Subway,43.704596,-79.349670,Sandwich Place,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
573,Thorncliffe Park,43.705369,-79.349372,Hakka Garden,43.704578,-79.349770,Indian Restaurant,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
574,Thorncliffe Park,43.705369,-79.349372,Iqbal foods,43.705751,-79.352054,Grocery Store,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio
575,Thorncliffe Park,43.705369,-79.349372,Petro-Canada,43.704058,-79.348094,Gas Station,2,Sandwich Place,Indian Restaurant,Supermarket,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Intersection,Yoga Studio


In [ ]:
### Cluster Four

In [74]:
df_Toronto_merged.loc[df_Toronto_merged['Cluster Labels'] == 3]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
930,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
931,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
932,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
933,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677702,-79.352265,Ice Cream Shop,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
934,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
935,"The Danforth West, Riverdale",43.679557,-79.352188,The Big Carrot Organic Juice Bar,43.677438,-79.352683,Juice Bar,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
936,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
937,"The Danforth West, Riverdale",43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
938,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677704,-79.350480,Greek Restaurant,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
939,"The Danforth West, Riverdale",43.679557,-79.352188,Louis Cifer Brew Works,43.677663,-79.351313,Brewery,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Japanese Restaurant,Indian Restaurant,Spa
